<h1 style="text-align:center"> Applications and workflow automation </h1>

In [1]:
from asetools.db import (get_session, DBAtom, System, SystemNote, atoms2system, get_atoms,
                         DBCalculator, calculator2db, ASETemplate)
from ase.visualize import view
import ase.io

In [3]:
dbpath = 'mytest.db'
session = get_session(dbpath)

# AseTemplate vs ASETemplate

In [4]:
from asetools import AseTemplate

In [5]:
dbtempate = session.query(ASETemplate).first()

In [6]:
dbtempate.name

u'geometry relaxation'

In [7]:
print dbtempate.template


#!/bin/env python

import os
import ase.io
from espresso import espresso
from ase.optimize import BFGS
from asetools import set_init_magmoms

atoms = ase.io.read('%atoms')

set_init_magmoms(atoms, [])

# Create vibration calculator
calc = espresso(pw=%pw,
                dw=%dw,
                xc = "BEEF-vdW",
                kpts = "gamma",
                sigma = 0.1,
                convergence={'energy':1e-10},
                spinpol=False,
                mode='scf', #single-point energy calculation per displacement
                )

atoms.set_calculator(calc)

# optimize geometry
qn = BFGS(atoms, logfile='geopt.log', trajectory='relaxed.traj')
qn.run(fmax=0.01)



In [8]:
atemp = AseTemplate(dbtempate.template)

## what to substitute

In [9]:
atemp.get_keys()

{'braced': [], 'escaped': [], 'invalid': [], 'named': [u'atoms', u'pw', u'dw']}

## render

In [10]:
atemp.render_and_write({'atoms' : 'myfile.xyz', 'pw' : 500, 'dw' : 7000}, output='myscript.py')

In [11]:
!cat myscript.py


#!/bin/env python

import os
import ase.io
from espresso import espresso
from ase.optimize import BFGS
from asetools import set_init_magmoms

atoms = ase.io.read('myfile.xyz')

set_init_magmoms(atoms, [])

# Create vibration calculator
calc = espresso(pw=500,
                dw=7000,
                xc = "BEEF-vdW",
                kpts = "gamma",
                sigma = 0.1,
                convergence={'energy':1e-10},
                spinpol=False,
                mode='scf', #single-point energy calculation per displacement
                )

atoms.set_calculator(calc)

# optimize geometry
qn = BFGS(atoms, logfile='geopt.log', trajectory='relaxed.traj')
qn.run(fmax=0.01)


# Reusing existing jobs to create new ones

Get all the `Systems` that are metal substituted zeotypes with acid hyrogen attach CO and write the geometry and job script to a directory

In [12]:
from asetools import attach_molecule, create_single_job
import os

In [13]:
systems = session.query(System).filter(System.name.like('%-H')).all()

create new jobs - each calculation in its own directory

read the CO geometry from file

In [14]:
co = ase.io.read('co.xyz')

In [15]:
os.mkdir('ZH-CO')
os.chdir('ZH-CO')

In [17]:
subs = {'pw' : 600, 'dw' : 7000, 'atoms' : 'initial.cif'}
for system in systems:
    atoms = get_atoms(session, system.id)
    aco = attach_molecule(atoms, 144, co, theta=-30, r=2.7)

    workdir = atoms.info['name'].replace('(', '_').replace(')', '') + '-CO'
    create_single_job(workdir, aco, system.template.template, subs, jobname='relax.py')
    submit.main(['relax.py', 'n', '2'])

/home/lmentel/.virtualenvs/cat/local/lib/python2.7/site-packages/ase/atoms.py:529: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if masses == 'defaults':


In [18]:
%ls -l

razem 328
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Ag_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Al_III-AFI-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 As_III-AFI-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Ba_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Be_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 B_III-AFI-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Br_III-AFI-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Ca_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Cd_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 C_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 C_IV-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Cl_III-AFI-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Co_II-AlPO-5-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Co_III-AFI-H-CO/
drwxr-xr-x 2 lmentel lmentel 4096 paź 21 11:58 Cr_II-AlPO-5-H-CO/
drwxr-xr-x 2 lm

In [19]:
%cd Zr_IV-AlPO-5-H-CO/

/home/lmentel/Devel/asetools/examples/ZH-CO/Zr_IV-AlPO-5-H-CO


In [20]:
atoms = ase.io.read('initial.cif')
view(atoms)

# Remote access

Run python notebook on a remote server (abel) through a tunnel and submit created jobs

In [1]:
!cat /home/lmentel/bin/tunnel2abel.sh

#!/bin/bash

LOGIN1=10.110.21.7
LOGIN2=10.110.21.8

ssh -N -f -L localhost:7777:localhost:7777 lukaszme@abel.uio.no


# Preprocessing &  analysis